In [11]:
import numpy as np
import pandas as pd
import polars as pl
import pytomlpp as toml
from data_preprocessing import getXY, data_normalizeation
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from tqdm import tqdm

In [12]:
# read config
cfg = toml.load('/home/gyli/GP_HP/cfg/feature_selection.toml')
data_path = cfg['data_path']
variance_filter_thres = cfg['variance_filter_thres']

In [13]:
raw_x, raw_y = getXY(data_path)
# raw_x = data_normalizeation(raw_x)
future_names = raw_x.columns
pd_x, pd_y = raw_x.to_pandas(), raw_y.to_pandas()

In [14]:
svm_clf = SVC()
X, Y = raw_x.to_pandas(), np.ravel(raw_y.to_pandas())

In [15]:
# filter by variance threshold
# from sklearn.feature_selection import VarianceThreshold
# var_thres = VarianceThreshold(threshold=variance_filter_thres)
# var_thres.fit(X)
# var_thres_col_idx = var_thres.get_support(indices=True)
# X_Var = X.iloc[:, var_thres_col_idx]
# print("\033[34m使用方差阈值筛选出来的特征结果索引:",var_thres.get_support(indices=True))
# print("\033[34m方差阈值选择前的结果:",cross_val_score(estimator=svm_clf,X=X,y=Y,cv=5,scoring='accuracy').mean())
# print("\033[34m方差阈值选择后的结果:",cross_val_score(estimator=svm_clf,X=X_Var,y=Y,cv=5,scoring='accuracy').mean())
X_Var = X

In [16]:
# 归一化
# X_Var = data_normalizeation(pl.from_pandas(X_Var))

In [17]:
list(X.columns)

['年龄',
 '中性粒细胞数',
 '淋巴细胞数',
 '嗜酸性粒细胞数',
 '嗜碱性粒细胞数',
 '总蛋白',
 '白蛋白',
 '球蛋白',
 '总胆红素',
 '直接胆红素',
 '钾',
 '钠',
 '钙',
 '尿素氮',
 '谷草谷丙',
 '甘油三酯',
 '高密度脂蛋白',
 '低密度脂蛋白',
 '红细胞平均体积',
 '凝血酶原时间',
 '国际标准化比值',
 '活化部分凝血活酶时间',
 '纤维蛋白原']

In [18]:
# filter by Chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
chi2_feature = SelectKBest(score_func=chi2, k='all')
chi2_selector = chi2_feature.fit(X_Var, Y)
chi2_selector.scores_
chi2_df = pd.DataFrame(np.arange(len(chi2_selector.scores_)), columns=['feature_idx'])
chi2_df['feature'] = list(X_Var.columns)
chi2_df['chi2_value'] = chi2_selector.scores_
sort_chi2 = chi2_df.sort_values(by='chi2_value', ascending=False).reset_index(drop=True)
print(sort_chi2)
X_Chi2_top10 = X_Var.iloc[:, sort_chi2[:10]['feature_idx'].to_list()]
X_Chi2_top14 = X_Var.iloc[:, sort_chi2[:14]['feature_idx'].to_list()]
print("\033[34m卡方选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Var,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m卡方选择top14后的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2_top10,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m卡方选择top10后的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2_top14,y=Y,cv=5,scoring='accuracy').mean())

    feature_idx     feature   chi2_value
0             6         白蛋白  3607.793828
1             5         总蛋白  2232.444816
2             0          年龄  1620.557003
3             1      中性粒细胞数  1262.392571
4            21  活化部分凝血活酶时间   383.776717
5            14        谷草谷丙   246.632228
6            15        甘油三酯   196.700687
7             8        总胆红素   102.868027
8             9       直接胆红素    90.602259
9             2       淋巴细胞数    89.572173
10           17      低密度脂蛋白    48.415306
11           18     红细胞平均体积    43.963457
12           19      凝血酶原时间    43.712427
13           11           钠    41.472782
14           13         尿素氮    22.948253
15           12           钙    17.311755
16            3     嗜酸性粒细胞数    15.833180
17           10           钾    14.112563
18            4     嗜碱性粒细胞数    10.461442
19           22       纤维蛋白原     3.729141
20           16      高密度脂蛋白     0.603448
21           20     国际标准化比值     0.559221
22            7         球蛋白     0.000358


卡方选择前的结果: 0.7198593923225771
卡方选择top14后的结果: 0.7238045203935901
卡方选择top10后的结果: 0.7202619607103495


In [19]:
X_Chi2 = X_Var.iloc[:, sort_chi2[:14]['feature_idx'].to_list()]

In [20]:
# 相关系数筛选
corr = X_Chi2.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool_))
to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]
print('deselect : ', to_drop)
upper

deselect :  ['总蛋白', '直接胆红素']


,白蛋白,总蛋白,年龄,中性粒细胞数,活化部分凝血活酶时间,谷草谷丙,甘油三酯,总胆红素,直接胆红素,淋巴细胞数,低密度脂蛋白,红细胞平均体积,凝血酶原时间,钠
白蛋白,NaN,0.821454,0.061810,0.460003,0.188782,0.334556,0.006366,0.211114,0.020222,0.214614,0.046843,0.077524,0.120331,0.356134
总蛋白,NaN,NaN,0.129596,0.338535,0.166047,0.256663,0.048175,0.153805,0.030848,0.196337,0.095966,0.024375,0.097896,0.244804
年龄,NaN,NaN,NaN,0.177365,0.131641,0.028208,0.138757,0.092833,0.104352,0.051793,0.075182,0.095411,0.169505,0.121082
中性粒细胞数,NaN,NaN,NaN,NaN,0.093932,0.147988,0.061344,0.111389,0.025697,0.038299,0.002683,0.054687,0.058327,0.351828
活化部分凝血活酶时间,NaN,NaN,NaN,NaN,NaN,0.117652,0.150532,0.066922,0.032863,0.031075,0.061671,0.174340,0.740851,0.126529
谷草谷丙,NaN,NaN,NaN,NaN,NaN,NaN,0.086485,0.109115,0.038425,0.141833,0.036579,0.007635,0.077541,0.122886
甘油三酯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.085004,0.085252,0.064004,0.024384,0.064693,0.217243,0.092386
总胆红素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.839410,0.006889,0.016982,0.049823,0.088564,0.004142
直接胆红素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059185,0.083656,0.022396,0.070193,0.070869
淋巴细胞数,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080059,0.025527,0.004276,0.131018


In [21]:
X_Corr = X_Chi2.drop(to_drop, axis=1)
print("Corr选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Chi2,y=Y,cv=5,scoring='accuracy').mean())
print("Corr选择后的结果:",cross_val_score(estimator=svm_clf,X=X_Corr,y=Y,cv=5,scoring='accuracy').mean())

Corr选择前的结果: 0.7202619607103495
Corr选择后的结果: 0.7186114133067856


In [22]:
# 随机森林筛选
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_Corr,Y)
feature_importance = rf_clf.feature_importances_
print(feature_importance)
feature_importance_df = pd.DataFrame(feature_importance, columns=['importance'])
feature_importance_df['feature'] = list(X_Corr.columns)
feature_importance_df['featrue_idx'] = np.arange(feature_importance_df.shape[0])
feature_importance_df = feature_importance_df.sort_values(ascending=False, by='importance').reset_index(drop=True)
print(feature_importance_df)
X_forest_top1 = X_Corr.iloc[:, feature_importance_df[:1]['featrue_idx'].to_list()]
X_forest_top6 = X_Corr.iloc[:, feature_importance_df[:6]['featrue_idx'].to_list()]
X_forest_top9 = X_Corr.iloc[:, feature_importance_df[:9]['featrue_idx'].to_list()]
X_forest_top11 = X_Corr.iloc[:, feature_importance_df[:11]['featrue_idx'].to_list()]
print("\033[34m随机森林选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Corr,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top1后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top1,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top6后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top6,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top9后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top9,y=Y,cv=5,scoring='accuracy').mean())
print("\033[34m随机森林选择top11后的结果:",cross_val_score(estimator=svm_clf,X=X_forest_top11,y=Y,cv=5,scoring='accuracy').mean())

[0.15597864 0.07628417 0.0863682  0.09470097 0.04597103 0.07342686
 0.06385917 0.0926456  0.09013985 0.06729913 0.07361221 0.07971418]
    importance     feature  featrue_idx
0     0.155979         白蛋白            0
1     0.094701  活化部分凝血活酶时间            3
2     0.092646       淋巴细胞数            7
3     0.090140      低密度脂蛋白            8
4     0.086368      中性粒细胞数            2
5     0.079714           钠           11
6     0.076284          年龄            1
7     0.073612      凝血酶原时间           10
8     0.073427        甘油三酯            5
9     0.067299     红细胞平均体积            9
10    0.063859        总胆红素            6
11    0.045971        谷草谷丙            4
随机森林选择前的结果: 0.7186114133067856
随机森林选择top1后的结果: 0.7138206801654963
随机森林选择top6后的结果: 0.723844705126697
随机森林选择top9后的结果: 0.7241669850658623
随机森林选择top11后的结果: 0.7191347546414176


In [23]:
X_Forest = X_Corr.iloc[:, feature_importance_df[:9]['featrue_idx'].to_list()]

In [24]:
# 支持向量机前向选择
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs1 = SFS(estimator=svm_clf,k_features=5,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
sfs1.fit(np.array(X_Forest), Y)
# SFS(estimator=SVC(), k_features=5, scoring='accuracy',verbose=2)

/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/home/gyli/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fi

In [ ]:
X_SFS1 = X_Forest.iloc[:, list(sfs1.k_feature_idx_)]
print("SFS前向选择前的结果:",cross_val_score(estimator=svm_clf,X=X_Forest,y=Y,cv=5,scoring='accuracy').mean())
print("SFS前向选择后的结果:",cross_val_score(estimator=svm_clf,X=X_SFS1.values,y=Y,cv=5,scoring='accuracy').mean())

SFS前向选择前的结果: 0.7241669850658623
SFS前向选择后的结果: 0.7264611444417546


In [ ]:
X_SFS1

NameError: name 'X_SFS1' is not defined

In [ ]:
# 暂使用X_SFS1训练SVM
gamma = cfg['gamma']
C = cfg['C']
best_score = 0.0
best_params = {'C' : -1.0, 'gamma' : -1.0}
for g in tqdm(gamma):
    for c in tqdm(C):
        svm = SVC(gamma=g, C=c)
        # svm.fit(X_SFS1, Y)
        scores = cross_val_score(svm, X_SFS1, Y, cv=5)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_params['C'] = c
            best_score['gamma'] = g
        tqdm.write(f'finish one param part, c = {c}, gamma = {g}, score = {score}')
tqdm.write(f'best param: C = {best_params["C"]}, gamma = {best_params["gamma"]}, score = {best_score}')


NameError: name 'cfg' is not defined